<a href="https://colab.research.google.com/github/Lukaid/my_notebook/blob/main/%EB%84%A4%EC%9D%B4%EB%B2%84_%EC%98%81%ED%99%94_%EB%A6%AC%EB%B7%B0%2C_%EB%89%B4%EC%8A%A4_%EC%8A%A4%ED%81%AC%EB%9E%98%ED%95%91.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 네이버 영화 리뷰 스크래핑

In [12]:
from bs4 import BeautifulSoup
import urllib.request
from urllib.parse import quote
import pandas as pd

def get_movie_reviews(mcode, page_num = 10):

    movie_review_df = pd.DataFrame(columns = ('Title', 'Score', 'Review'))
    url = f"https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword={mcode}&target=after"
    idx = 0

    for _ in range(0, page_num):
        movie_page = urllib.request.urlopen(url).read()
        movie_page_soup = BeautifulSoup(movie_page, 'html.parser')

        review_list = movie_page_soup.find_all('td', attrs = {'class' : 'title'})
        for review in review_list:
            title = review.find('a', attrs = {'class' : 'movie color_b'}).get_text()
            score = review.find('em').get_text()
            review_text = review.find('a', attrs = {'class' : 'report'}).get('onclick').split(',')[2]
            movie_review_df.loc[idx] = [title, score, review_text]
            idx += 1
            print('#', end = "")
        try:
            url = 'https://movie.naver.com' + movie_page_soup.find('a',{'class' : 'pg_next'}).get('href')
        except:
            break
    
    return movie_review_df

In [15]:
get_movie_reviews(18847, 10)

####################################################################################################

,Title,Score,Review
0,타이타닉,10,'진짜 나한테 인생 영화라는게 없었는데 이제 생겼음 안보면 진짜 후회 합니다 몇번...
1,타이타닉,10,'선상키스신은 정말.. 하 연애 하고 싶다!'
2,타이타닉,9,'사랑과 재난의 비극적인 조화
3,타이타닉,10,'죽기전에 한편만 다시 볼수 있다면 기꺼이 선택할 영화'
4,타이타닉,10,'고3때보고 마흔이 넘은 지금봐도 눈물나네 썅'
...,...,...,...
95,타이타닉,10,'불필요한 장면이 없는 명작'
96,타이타닉,10,'타이타닉을 본건 내 생의최고의영광'
97,타이타닉,10,'2021년에 봐도 재밌네요.... 가슴이 흐어.. 줵 - 컴백 ㅠ_ㅠ!!! ...'
98,타이타닉,10,'슬픔너무슬픔진슬픔ㅋㅋㅋ'


## 현재 상영작 영화 리뷰 스크래핑

In [17]:
url = 'https://movie.naver.com/movie/point/af/list.nhn'
naver_movie = urllib.request.urlopen(url).read()
soup = BeautifulSoup(naver_movie, 'html.parser')
select = soup.find('select', {'id' : 'current_movie'})
movies = select.find_all('option')

movies_dict = {}
for movie in movies[1:]:
    movies_dict[movie.get('value')] = movie.get_text()

movie_review_df = pd.DataFrame(columns = ('Title', 'Score', 'Review'))
for mcode in movies_dict:
    df = get_movie_reviews(mcode, 1)
    movie_review_df = pd.concat([movie_review_df, df])

movie_review_df

########################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################

,Title,Score,Review
0,명탐정 코난: 비색의 탄환,8,'진짜 캐릭터 감정 잘 나타나고 손에 땀을 쥐게함ㅋㅋㅋㅋㄹㅇ로 뻥안치고 2시간 순...
1,명탐정 코난: 비색의 탄환,10,'역대급입니다 캬 빌드업 대성공 '
2,명탐정 코난: 비색의 탄환,10,'스토리에맞는 사운드가 엄청 다채롭고 풍부했어요 소재도 좋았고 늘어짐없는 재밋는...
3,명탐정 코난: 비색의 탄환,10,'4dx 추천박음 다때리뿌수는데 ㄹㅇ재밌어요'
4,명탐정 코난: 비색의 탄환,6,'솔직히 기대했는데 별로였습니다. 다른 극장판과는 다르게 너무 유치하다고 해야하나...
...,...,...,...
5,빅 컨츄리,4,'50년대 서부극은 &amp;#39;수색자&amp;#39;
6,빅 컨츄리,10,'러닝타임 끝무렵까지 등장인물중 아무도 죽는 이 없는 그러나 팽팽한 긴장의 끈을 ...
7,빅 컨츄리,8,'대형 스크린으로 다시 보고 싶다...'
8,빅 컨츄리,10,'전쟁이란.. 노인이 결정하고 젊은이들이 피흘리고 죽는것이라는데


# 네이버 뉴스 기사 스크래핑

In [68]:
from bs4 import BeautifulSoup
import urllib.request
from urllib.parse import quote
import pandas as pd



def get_news(query, page_num = 10):

    news_df = pd.DataFrame(columns=('Title', 'Link', 'Datetime', 'Article'))
    idx = 0

    for _ in range(page_num):

        url_query = quote(query)
        url = f"https://search.naver.com/search.naver?query={url_query}&where=news&ie=utf8&sm=nws_hty"

        search_url = urllib.request.urlopen(url).read()
        soup = BeautifulSoup(search_url, 'html.parser')
        links = soup.find_all('div', {'class' : 'info_group'})

        for link in links:
            try:
                news_url = link.find_all('a', {'class':'info'})[1].get('href')
                news_link = urllib.request.urlopen(news_url).read()
                news_html = BeautifulSoup(news_link, 'html.parser')

                title = news_html.find('h3', {'id' : 'articleTitle'}).get_text()
                datetime = news_html.find('span', {'class' : 't11'}).get_text()
                article = news_html.find('div', {'id' : 'articleBodyContents'}).get_text()
                article = article.replace('// flash 오류를 우회하기 위한 함수 추가', '')
                article = article.replace('내가 지우고 싶은 문자들 입력', '')

                news_df.loc[idx] = [title, news_url, datetime, article]
                idx += 1
                print('#', end = "")

            except:
                continue

        try:  
            next = soup.find('a', {'class':'btn_next'}).get('href')
            url = 'https://search.naver.com/search.naver' + next
        except:
            break

    return news_df



get_news('쿠팡')

,Title,Link,Datetime,Article
